In [2]:
import tensorflow as tf

In [3]:
import os
import numpy as np
import cv2
from skimage.io import imread, imshow
from skimage.transform import resize

In [9]:
from tensorflow import keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
from datetime import datetime

In [10]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [11]:
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [25]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = [1,1]

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        sd.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

In [26]:
history=LossHistory()

In [29]:
sd=[]

In [22]:
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(losses):
    if float(2*np.sqrt(np.array(history.losses[-1])))<0.3:
        lrate=0.01*1/(1+0.1*len(history.losses))
        momentum=0.8
        decay_rate=2e-6
        return lrate
    else:
        lrate=0.1
        return lrate
lrate=LearningRateScheduler(step_decay)


In [15]:
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)


In [13]:
%load_ext tensorboard
log_dir="/space/storage/homes/vrv/cellseg-cuda/victor/logs/"
tensorboard_callback= tf.keras.callbacks.TensorBoard(log_dir=log_dir,                     
                               histogram_freq=1)

In [6]:
model = tf.keras.models.load_model('saved_model/3_class_50ep_after_new_data_SGD')

In [16]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x

def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)

if __name__ == "__main__":
    model = build_unet((128, 128, 1), 3)
    #model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
X_paths = os.listdir('/space/storage/homes/vrv/cellseg-cuda/processed_data/new_data/originals')
y_paths = os.listdir('/space/storage/homes/vrv/cellseg-cuda/processed_data/new_data/labels_3_classes')
X_paths.sort()
y_paths.sort()
nb_img=len(X_paths)
img_height=128
img_width=128
X = np.zeros((nb_img, img_height, img_width,1), dtype=np.uint8)
y = np.zeros((nb_img, img_height, img_width,1), dtype=np.uint8)
for i in range(len(X_paths)) : 
    img = imread('/space/storage/homes/vrv/cellseg-cuda/processed_data/new_data/originals/' + X_paths[i])
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis=-1)
    X[i]=img
for j in range(len(y_paths)) : 
    img = imread('/space/storage/homes/vrv/cellseg-cuda/processed_data/new_data/labels_3_classes/' + y_paths[j])
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis=-1)
    y[j]=img
n = len(X)
r = int(n*0.8)

X_train = X[:r]
X_test = X[r+1:]

y_train = y[:r]
y_test = y[r+1:]

In [ ]:
X_train.shape,y_tr

In [7]:
print(len(X),len(y))

66181 66181


In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ["accuracy"])

In [9]:
from tensorflow import keras as keras
tbCallBack = keras.callbacks.TensorBoard(log_dir='victor/tbCb3', histogram_freq=0, write_graph=True, write_images=True)

In [15]:
checkpoint_path = "model/cpNEW_DATA.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

### CALLBACKS

In [18]:
model.compile(optimizer=opt
, loss='sparse_categorical_crossentropy', metrics = ["accuracy"])

In [19]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [31]:
model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=50, callbacks=[tensorboard_callback, mc, es, history, lrate],verbose=1)

Train on 47649 samples, validate on 5295 samples
Epoch 1/50
47649/47649 [==============================] - 89s 2ms/sample - loss: 0.0443 - acc: 0.9817 - val_loss: 0.0407 - val_acc: 0.9832
Epoch 2/50
47649/47649 [==============================] - 89s 2ms/sample - loss: 0.0424 - acc: 0.9825 - val_loss: 0.0360 - val_acc: 0.9851
Epoch 3/50
47649/47649 [==============================] - 89s 2ms/sample - loss: 0.0407 - acc: 0.9832 - val_loss: 0.0354 - val_acc: 0.9848
Epoch 4/50
47649/47649 [==============================] - 88s 2ms/sample - loss: 0.0396 - acc: 0.9837 - val_loss: 0.0356 - val_acc: 0.9851
Epoch 5/50
47649/47649 [==============================] - 88s 2ms/sample - loss: 0.0384 - acc: 0.9842 - val_loss: 0.0445 - val_acc: 0.9820
Epoch 6/50
12144/47649 [======>.......................] - ETA: 1:03 - loss: 0.0376 - acc: 0.9845

KeyboardInterrupt: 

In [12]:
model.save('saved_model/3_class_50ep_after_new_data_adam')

In [20]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)


Evaluate on test data
13236/13236 [==============================] - 6s 446us/sample - loss: 0.0488 - acc: 0.9874
test loss, test acc: [0.04884987081414841, 0.98736703]


### Test sur une image (289 premiers)

In [3]:
from os import listdir
from os.path import isfile, join
img_height=128
img_width=128
dest_originals_path='/space/storage/homes/vrv/cellseg-cuda/processed_data/new_data/originals/'
d_o = [f for f in listdir(dest_originals_path) if isfile(join(dest_originals_path, f))]
d_o.sort()
test_predict=[]
for i in range(289) : 
    img = imread(dest_originals_path + d_o[i])
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis=-1)
    test_predict.append(img)
test_predict=np.array(test_predict)
test_predict.shape

(289, 128, 128, 1)